In [17]:
import xarray as xr
import fsspec
import s3fs
import os
import matplotlib.pyplot as plt
import dask
import rasterio
from dask.distributed import Client, LocalCluster, progress
import datetime
import tempfile
import boto3

In [2]:
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
           AWS_NO_SIGN_REQUEST='YES',
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
           GDAL_SWATH_SIZE='200000000',
           VSI_CURL_CACHE_SIZE='200000000')
os.environ.update(env)

In [26]:
goes17_bucket = "goes17"

In [3]:
cluster = LocalCluster(processes=False, local_directory='/tmp') 
client = Client(cluster) 

In [42]:
def convert_full_date_to_continous_day(year, month, day):
    return datetime.datetime(year, month, day).timetuple().tm_yday

def get_geo_data(year, month, day):
    fs = s3fs.S3FileSystem(anon=True)
    continous_day = convert_full_date_to_continous_day(year, month, day)
    filepath = "s3://noaa-goes17/ABI-L2-SSTF/%s/%s/*/*.nc" % (str(year).zfill(4), str(day).zfill(3)) 
    files, var = fs.glob(filepath), 'SST'
    
    if len(files) < 1:
        raise Exception("No files found")


    file_ob = [fs.open(file) for file in files]
    ds = xr.open_mfdataset(file_ob, combine='nested', concat_dim='time')
    
    return ds


In [43]:
d = get_geo_data(2020, 11, 8)